In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import sys,time, random,os
from time import sleep
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas


# PROXY = "139.255.17.2:47421" # IP:PORT or HOST:PORT

options = Options()
# options.add_argument("--headless")
# options.add_argument('--proxy-server=%s' % PROXY)
# options.headless = True
driver = webdriver.Chrome('chromedriver.exe',options=options)

url = "https://www.yelp.com/search?cflt=nightlife&find_loc=San+Francisco%2C+CA"

driver.get(url)
sleep(5)
html = driver.page_source
lists = driver.find_elements_by_css_selector('.alternate__373c0__1uacp')

Ads =[]
Ads_links = []
normal = []
normal_links = []

def get_links(lists):
    for i in lists:
        if "Ad" in i.text:
            try:
                if i.find_element_by_css_selector('.link-size--inherit__373c0__2JXk5').text in Ads:
                    continue
                else:
                    Ads.append(i.find_element_by_css_selector('.link-size--inherit__373c0__2JXk5').text)
                    Ads_links.append(i.find_element_by_css_selector('.link-size--inherit__373c0__2JXk5').get_attribute('href'))
            except:
                continue
        else:
            try:
                if i.find_element_by_css_selector('.link-size--inherit__373c0__2JXk5').text in normal:
                    continue
                else:
                    normal.append(i.find_element_by_css_selector('.link-size--inherit__373c0__2JXk5').text)
                    normal_links.append(i.find_element_by_css_selector('.link-size--inherit__373c0__2JXk5').get_attribute('href'))
            except:
                continue

def get_next_page_links():
    try:
        next_link = driver.find_element_by_link_text('Next')
        next_url = next_link.get_attribute('href')  # if removed
        driver.get(next_url)
        next_page = driver.page_source
        lists = driver.find_elements_by_css_selector('.alternate__373c0__1uacp')
    except:
        print("no link found")
        lists[:] = []
    return lists

def link_extraction(links):
    site_urls = []
    for link in links:
        text = ""
        driver.get(link)
        page_html = driver.page_source
        try:
            map_box = driver.find_element_by_css_selector('.mapbox-text')
            elements = map_box.find_elements_by_css_selector('a')
            verify = False
            for i in elements:
                if '.com' in i.text:
                    text = i.text
                    verify = True
                    site_link = driver.find_element_by_link_text(text).get_attribute('href')
                    site_urls.append(site_link)
            if not verify:
                site_urls.append("No Site URL Present")
        except:
            print("found an exception")
            site_urls.append("Found an exception at : "+ link)
            continue
    return site_urls

def extract_url(urls):
    url_list = []
    for url in urls:
        print(url)
        if "No Site URL Present" in url or "Found an exception at : " in url:
            url_list.append(url)
        else:
            site = url.split('&')[0].split('=')[1].replace('%3A',':').replace('%2F','/')
            url_list.append(site)
            print(site)
    return url_list

def convert_to_file(list_of_links,name,column_name):
    df = pandas.DataFrame(list_of_links,columns = [column_name],index = None)
    df.to_csv(name,index = None)

while True:
    get_links(lists)
    lists = get_next_page_links()
    if len(lists) == 0:
        break

Ads_list = list(Ads_links)
convert_to_file(Ads_list,'AP_file.csv','Link')

normal_list = list(normal_links)
convert_to_file(normal_list,'NP_file.csv','Link')

print("Page links fetched")

Ads_URLS = link_extraction(Ads_links)
APU = extract_url(Ads_URLS)
convert_to_file(Ads_site_URLS,'APU.csv','Site URL')

print("APU Done")

Normal_URLS = link_extraction(normal_links)
NPU = extract_url(Normal_URLS)
convert_to_file(NPU,'NPU.csv','Site URL')

print("NPU Done")

driver.close()

Page links fetched
https://www.yelp.com/biz_redir?url=https%3A%2F%2Fsfbaydjs.com%2Fbest-san-francisco-wedding-party-dj%2F&website_link_type=website&src_bizid=QjL22rH2CT6noMmennkfxQ&cachebuster=1544126745&s=42389b60c518b1cb5dab5b6007602ed916f00bd33108c4494dfc92158056428e&campaign_id=p8OxgLatNuJHqtzzkL_Wxw
https://sfbaydjs.com/best-san-francisco-wedding-party-dj/
https://www.yelp.com/biz_redir?url=http%3A%2F%2Fthelaundrysf.com&website_link_type=website&src_bizid=NIoaaJaMkeYmQjLs9a0D2A&cachebuster=1544126755&s=a14e4c9558c87523b035101263b6b494674002d7413c25a291735efc48ce86ba&campaign_id=Auj0qSYqgt-T5Gd0HhA2Rw
http://thelaundrysf.com
https://www.yelp.com/biz_redir?url=http%3A%2F%2Fwww.gilberths.com%2F&website_link_type=website&src_bizid=kT-g8Z5A7o0tatjap4ghAw&cachebuster=1544126764&s=69adb5d529a5a575e6bd5ecb87b69ce5aad23ad667e8ace05e93d14948950e9e&campaign_id=F2l5lkqHUScuMDMLxUEmWw
http://www.gilberths.com/
https://www.yelp.com/biz_redir?url=http%3A%2F%2Fwww.poplarsm.com&website_link_type=w

https://www.yelp.com/biz_redir?url=http%3A%2F%2FBootieSF.com&website_link_type=website&src_bizid=_KsiqRiYtmRaqltOA96QZw&cachebuster=1544127710&s=1834b4896047e0070ba3c7a37577f8481821e0f445cda9f7d0c5ba001fd6a262
http://BootieSF.com
No Site URL Present
https://www.yelp.com/biz_redir?url=https%3A%2F%2Fwww.marriott.com%2Fhotels%2Fhotel-information%2Frestaurant%2Fsfodt-san-francisco-marriott-marquis%2F&website_link_type=website&src_bizid=lOCewxz6QNegZbArImNBfw&cachebuster=1544127724&s=a3f83678b24c246cb8adc6ab0c4439f5d93a720bf3599af65e7b463f4a0ac749
https://www.marriott.com/hotels/hotel-information/restaurant/sfodt-san-francisco-marriott-marquis/
https://www.yelp.com/biz_redir?url=http%3A%2F%2Ffeightsf.com&website_link_type=website&src_bizid=FIX6Ea02Hx2bO_3z4kj64g&cachebuster=1544127730&s=55856807bbead619e8265d45950ea29339d4c1916e9e4dcce0c1bb39580386af
http://feightsf.com
No Site URL Present
https://www.yelp.com/biz_redir?url=http%3A%2F%2Fwww.lastnite00s.com&website_link_type=website&src_bizi

No Site URL Present
https://www.yelp.com/biz_redir?url=http%3A%2F%2Fwww.pig-and-whistle.com&website_link_type=website&src_bizid=eozQsh2UnHM9aok46zGgyA&cachebuster=1544131730&s=066d6d7e2961781a2996d620853049ca10b3803090f3c3cc7515e57e1c29a838
http://www.pig-and-whistle.com
https://www.yelp.com/biz_redir?url=https%3A%2F%2Fthrillcall.com&website_link_type=website&src_bizid=Vv7XLcJf1MfQFkI61K8qZQ&cachebuster=1544131738&s=6bacc7c30c2fbb157f79bb8825bcc96660b924d2943ade2d29b6efada7e95bd4
https://thrillcall.com
https://www.yelp.com/biz_redir?url=http%3A%2F%2Fbrickandmortarmusic.com%2F&website_link_type=website&src_bizid=6s67w_r3LZXiSZG8URUq8w&cachebuster=1544131744&s=c13f69249528a951aebc84a886a671d57ec74ca8dd2fd16580f2cfa665750b13
http://brickandmortarmusic.com/
https://www.yelp.com/biz_redir?url=http%3A%2F%2Fwww.thelisterbar.com&website_link_type=website&src_bizid=e6V6AK0YxmYRIGGDCdnnww&cachebuster=1544131749&s=c79d8e5ca04cd3e44e957e0ea2ef0f42a0395658d363d387fccd09c05c64c1b8
http://www.thelist